## Installing libraries used for langchain and for google api key

# Prompt of generated video from Runway ML
### "Generate a 30-second animated video illustrating the journey of a butterfly. Start with the butterfly emerging from its chrysalis, followed by scenes of it exploring flowers, and conclude with the butterfly flying into a sunset. Use vibrant, naturalistic visuals."


In [ ]:
!pip install -q -U langchain
!pip install -q -U google-genai
!pip install -q -U langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.3 MB/s eta 0:00:00


## importing api key from secrets

In [ ]:
from google.colab import userdata
import os
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain.schema import SystemMessage
import json
import os

In [ ]:

def analyze_video_with_langchain(video_path):
    """Analyzes an AI-generated video using Langchain and Gemini."""

    # 1.  Initialize the Model
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp", google_api_key=os.environ["GOOGLE_API_KEY"])

    # 2. Define the Prompt
    system_template = """
    You are an expert video analyzer. Your task is to describe scenes from a video.
    Pay attention to key visual elements and any spoken words.
    The provided video is AI generated, so there might be some minor inconsistencies.
    """
    human_template = """
     For each distinct scene in the provided video, generate a caption.
     Include the approximate starting timecode (in seconds) of the scene, a detailed description of the visual elements in the scene,
     and any spoken text present in that scene.
     Output your response as a JSON array of objects, where each object represents a scene with the following keys:

        "timecode_seconds": (float),
        "description": (string),
        "spoken_text": (string).
     If there is no spoken text, set the value of "spoken_text" to null.

    Do not use any preambles or other information outside the JSON format.

    """
    prompt = ChatPromptTemplate.from_messages(
        [
           SystemMessage(content=system_template),
            ("human", "{video_path} " + human_template), # Pass video path to the prompt
        ]
    )
    parser = JsonOutputParser()


    # 3. Create a Chain (using the Prompt, Parser, and LLM)
    chain = prompt | llm | parser

    # 4. Run the chain with the video path
    try:
       output = chain.invoke({"video_path": video_path})
       #print(json.dumps(output, indent=4))  #Pretty print the output

       return output #Return the parsed output
    except Exception as e:
       print(f"An error occurred: {e}")
       return None



## Giving path of the saved video


In [ ]:
if __name__ == "__main__":


  video_file_path = "/content/butterfly_journey.mp4"


  analysis_result = analyze_video_with_langchain(video_file_path)

  if analysis_result:
        print("Video Analysis Results:")
        print(json.dumps(analysis_result, indent=4))
  else:
       print("Video analysis failed")

Video Analysis Results:
[
    {
        "timecode_seconds": 0.0,
        "description": "The video opens with a close-up shot of a monarch butterfly caterpillar munching on a green leaf. The caterpillar is green with black and yellow stripes. The leaf is textured and has visible veins. The background is blurred, suggesting a shallow depth of field.",
        "spoken_text": "null"
    },
    {
        "timecode_seconds": 2.9,
        "description": "The camera zooms out slightly to reveal more of the plant. The caterpillar continues to eat the leaf. The background is still out of focus but appears to be more leaves and foliage.",
        "spoken_text": "null"
    },
    {
        "timecode_seconds": 5.8,
        "description": "The scene transitions to a different shot of the caterpillar, now hanging upside down in a J-shape, preparing to pupate. It is attached to a thin stem or branch. The background is a mix of blurry green and brown.",
        "spoken_text": "null"
    },
    {
     